### loop over images in `/sample/original` to crop them to just the gate

In [ ]:
from PIL import Image
import glob

original_jpg_files = glob.glob('./sample/original/*.jpg')

for index, file in enumerate(original_jpg_files):
    # Process each file here
    im = Image.open(file)

    # Size of the image in pixels (size of original image)
    # (This is not mandatory)
    width, height = im.size

    left = 2142
    top = 333
    right = left + 585
    bottom = top + 468

    # Cropped image of above dimension
    im1 = im.crop((2142, 333, right, bottom))

    # save the cropped image in `cropped` folder, create folder if necessary
    im1.save(file.replace('original', 'cropped'))

    print("processed {} of {}".format(index + 1, len(original_jpg_files)))


### send to Roboflow

In [ ]:
import os
from roboflow import Roboflow
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

# Initialize the Roboflow object with your API key
rf = Roboflow(api_key=os.environ.get("ROBOFLOW_API_KEY"))

# Retrieve your current workspace and project name
print(rf.workspace())

# Specify the project for upload
project = rf.workspace("long-zheng-bzpnl").project("gate-open-or-closed")

cropped_jpg_files = glob.glob('./sample/cropped/*.jpg')

for index, file in enumerate(cropped_jpg_files):
    # Upload the image to your project
    project.upload(file)

    print("uploaded {} of {}".format(index + 1, len(cropped_jpg_files)))

### download dataset from roboflow

In [ ]:
import os
from roboflow import Roboflow
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

rf = Roboflow(api_key=os.environ.get("ROBOFLOW_API_KEY"))
project = rf.workspace("long-zheng-bzpnl").project("gate-open-or-closed")
dataset = project.version(5).download("folder")

# rename dataset /valid to /val
import shutil
shutil.move(dataset.location + "/valid", dataset.location + "/val")


### train data

In [ ]:
!yolo task=classify mode=train model=yolov8n-cls.pt data='Gate-open-or-closed-5' epochs=100

### upload weights to roboflow

In [ ]:
import os
from roboflow import Roboflow
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

rf = Roboflow(api_key=os.environ.get("ROBOFLOW_API_KEY"))
project = rf.workspace("long-zheng-bzpnl").project("gate-open-or-closed")
version = project.version(5)
version.deploy("yolov8-cls", "runs/classify/train/")

### validate data

In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO('runs/classify/train/weights/best.pt')

# OR define a recursive glob search for all JPG files including subdirectories
source = 'Gate-open-or-closed-5/test/**/*.jpg'

# Run inference on the source
results = model(source, stream=True,)  # generator of Results objects

for result in results:
    print(result.probs.top1)

### export ONNX nodel

In [ ]:
!yolo export model=runs/classify/train/weights/best.pt format=onnx